# 3주차 Day3 — 출처 정확성 검증 데모

> **블록3 심화용** | 강사 전용 데모 노트북
>
> 블록1에서 `ask("질문")`으로 답변을 봤다면,
> 이번에는 **출처가 정말 맞는지** PDF를 열어서 대조합니다.
>
> **강의 전 준비**: 아래 `[준비]` 셀 2개를 미리 실행해두세요.

---
## [준비] 시스템 초기화
> 강의 전에 미리 실행. 수강생 화면에는 보여주지 않음.

In [ ]:
# ============================================================
# [준비 1/2] 패키지 설치 + 환경설정
# 강의 전에 미리 실행해 두세요. 수강생 화면에는 보여주지 않습니다.
# ============================================================


import os  # 파이썬 기본 라이브러리 - 환경 변수(시스템 설정값) 조작에 사용


# -----------------------------------------------------------
# TensorFlow + protobuf 충돌 방지 설정
# -----------------------------------------------------------
# 이 환경에는 TensorFlow(구글 AI 프레임워크)와
# PyTorch(메타 AI 프레임워크)가 모두 설치되어 있습니다.
# 두 프레임워크가 공유하는 protobuf 라이브러리 버전이 맞지 않으면
# 실행 즉시 오류가 납니다.
# 해결: 아래 환경 변수로 TensorFlow를 꺼두고 PyTorch만 사용합니다.
# ★ 반드시 다른 라이브러리 import 보다 먼저 실행해야 합니다! ★
os.environ["USE_TF"] = "0"                           # TensorFlow 비활성화 (충돌 방지 핵심)
os.environ["USE_TORCH"] = "1"                        # PyTorch 활성화 (HuggingFace 임베딩에 필요)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"             # TensorFlow 경고 메시지 숨기기 (3=모두 숨김)
os.environ["TOKENIZERS_PARALLELISM"] = "false"       # 토크나이저 병렬 처리 관련 경고 억제
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"  # Windows 심링크 경고 숨기기


# -----------------------------------------------------------
# HuggingFace 다운로드 서버 우회 (방화벽 환경 대응)
# -----------------------------------------------------------
# 회사/학교 네트워크에서 huggingface.co 원본 서버가 차단된 경우,
# hf-mirror.com 미러 서버를 통해 AI 모델을 내려받습니다.
# (미러 서버 = 같은 내용을 복사해 둔 백업 서버)

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# -----------------------------------------------------------
# 필요한 파이썬 패키지 설치
# -----------------------------------------------------------
# !pip install: 외부 라이브러리를 인터넷에서 내려받아 설치하는 명령
# -q 옵션: 설치 로그를 최소화 (quiet 모드). 이미 설치된 패키지는 자동 스킵.


!pip install -q langchain langchain-community langchain-huggingface langchain-text-splitters chromadb pypdf sentence-transformers


#   설치되는 패키지 설명:
#   langchain               : AI 부품들을 레고처럼 연결하는 파이프라인 프레임워크
#   langchain-community     : PDF 로더, ChromaDB 연결 등 커뮤니티 제공 확장 도구
#   langchain-huggingface   : HuggingFace 임베딩 모델을 LangChain에 연결하는 어댑터
#   langchain-text-splitters: 긴 문서를 일정 크기로 잘라주는 청킹 도구
#   chromadb                : 벡터(숫자 배열)를 저장하고 빠르게 검색하는 벡터 데이터베이스
#   pypdf                   : PDF 파일에서 텍스트를 추출하는 라이브러리
#   sentence-transformers   : 문장을 384차원 숫자 벡터로 변환하는 경량 AI 모델 (약 90MB)


from IPython.display import display, HTML  # Jupyter 노트북에서 HTML 형식으로 결과 출력


# -----------------------------------------------------------
# Google Gemini API 키 설정
# -----------------------------------------------------------
# Gemini: 구글이 만든 대형 언어 모델(LLM). 텍스트를 이해하고 답변을 생성합니다.
# API 키: AI 서비스를 사용하기 위한 인증 코드 (신분증 역할)
#         aistudio.google.com -> Get API key 에서 무료 발급 가능
# 아래 문자열을 본인의 API 키로 교체하세요.


os.environ["GOOGLE_API_KEY"] = "AIzaSy...여기에_새_API_키_입력"  # <- 여기 교체


# .env 파일에 API 키를 저장해 둔 경우 자동으로 불러옵니다 (없어도 무관)

try:
    from dotenv import load_dotenv
    load_dotenv()                       # 현재 폴더의 .env 파일에서 환경 변수 읽기
except ImportError:
    pass                                # python-dotenv가 없으면 이 단계를 건너뜁니다

# API 키가 비어 있으면 즉시 오류를 내서 미리 알려줍니다

assert os.environ.get("GOOGLE_API_KEY"), "GOOGLE_API_KEY가 설정되지 않았습니다!"
print("환경설정 완료")

In [ ]:
# ============================================================
# [준비 2/2] 벡터DB 구축 + RAG 체인 생성
# RAG(Retrieval-Augmented Generation) 파이프라인 전체를 준비합니다.
# 강의 전에 미리 실행해 두세요. 수강생 화면에는 보여주지 않습니다.
# ============================================================

# --- 필요한 라이브러리 불러오기 ---


import os
from pathlib import Path                                             # 파일/폴더 경로 처리
from langchain_community.document_loaders import PyPDFLoader         # PDF -> 텍스트 변환
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_community.vectorstores import Chroma                  # 벡터 데이터베이스
from langchain_huggingface import HuggingFaceEmbeddings              # 로컬 임베딩 모델
from langchain_core.prompts import PromptTemplate                    # 프롬프트 템플릿
from google import genai as google_genai                             # Google Gemini SDK
from IPython.display import display, HTML                            # HTML 출력


# -----------------------------------------------------------
# Step 0. PDF 파일 위치 자동 탐색
# -----------------------------------------------------------
# 노트북을 어느 폴더에서 실행하든 PDF 파일을 찾아서 작업 폴더를 자동으로 변경합니다.
# os.walk(): 지정 폴더 아래의 모든 하위 폴더를 재귀적으로 순회
#   - root  : 현재 탐색 중인 폴더 경로 (문자열)
#   - dirs  : 그 폴더 안의 하위 폴더 이름 목록
#   - files : 그 폴더 안의 파일 이름 목록


pdf_name = "제품사양서_스마트냉장고_RF9000.pdf"
for root, dirs, files in os.walk(str(Path.cwd())):
    if pdf_name in files:                                  # 찾는 PDF가 있는 폴더를 발견하면
        os.chdir(root)                                     # 그 폴더를 작업 디렉토리로 변경
        print(f"작업 디렉토리: {root}")
        break                                              # 탐색 종료


# -----------------------------------------------------------
# Step 1. PDF 로딩 - 문서를 파이썬이 읽을 수 있는 형태로 변환
# -----------------------------------------------------------
# PyPDFLoader: PDF 파일을 열어서 페이지별로 텍스트를 추출합니다.
# 각 페이지는 Document 객체로 변환됩니다.
#   Document.page_content : 페이지의 텍스트 내용
#   Document.metadata     : {"source": "파일명.pdf", "page": 0}  (page는 0부터 시작)

          
pdf_files = [
    "제품사양서_스마트냉장고_RF9000.pdf",             # 제품 스펙 문서 (4페이지)
    "시험성적서_스마트냉장고_RF9000.pdf"              # 공인 시험 결과 문서 (3페이지)
]
all_docs = []                                      # 모든 페이지 Document를 담을 빈 리스트
for pdf_path in pdf_files:
    loader = PyPDFLoader(pdf_path)                 # PDF 읽기 도구 생성
    docs = loader.load()                           # PDF를 읽어서 페이지별 Document 리스트 반환
    all_docs.extend(docs)                          # 전체 리스트에 추가 (extend = 리스트 이어붙이기)
    print(f"{pdf_path} -> {len(docs)}페이지 로딩")
print(f"총 {len(all_docs)}페이지 로딩 완료")          # 정상이면 7이 출력됩니다


# ------------------------------------------------
# -----------
# Step 2. 청킹 - 긴 문서를 작은 조각(청크)으로 분할
# -----------------------------------------------------------
# 문제: 한 페이지 전체를 AI에 넘기면 너무 길어서 핵심을 못 찾습니다.
# 해결: 500자씩 잘라서 검색 정확도를 높입니다.
#
# RecursiveCharacterTextSplitter: 문단 -> 문장 -> 단어 순으로 자연스럽게 분할
# chunk_size=500   : 한 조각의 최대 글자 수
# chunk_overlap=50 : 인접 조각 사이에 50자를 겹침 -> 문맥이 끊기지 않도록 연결


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(all_docs)           # 7페이지 -> 여러 개의 작은 조각으로 분할
print(f"{len(chunks)}개 청크로 분할")


# -----------------------------------------------------------
# Step 3. 벡터DB 구축 - 텍스트를 숫자 벡터로 변환하여 저장
# -----------------------------------------------------------
# 임베딩(Embedding)이란?
#   텍스트를 수백 개의 숫자로 이루어진 배열(벡터)로 변환하는 과정입니다.
#   의미가 비슷한 문장은 비슷한 벡터를 가집니다.
#   예) "냉장고 용량" <-> "냉장고 크기"  : 매우 가까운 벡터 (유사)
#       "냉장고 용량" <-> "오늘 날씨"    : 매우 먼 벡터 (무관)
#
# HuggingFaceEmbeddings: 인터넷 서버 없이 로컬에서 실행하는 경량 임베딩 모델
#   model_name: all-MiniLM-L6-v2 = 성능/속도 균형이 좋은 384차원 모델 (약 90MB)
#   device="cpu": GPU 없이 CPU만으로 실행


print("임베딩 모델 로딩 중...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)


# Chroma.from_documents: 각 청크를 임베딩 벡터로 변환 후 DB에 일괄 저장


vectorstore = Chroma.from_documents(chunks, embeddings)


# retriever: 질문을 받아서 가장 유사한 청크 k개를 반환하는 검색기 객체

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # 상위 3개 반환
print("벡터DB 구축 완료")

# -----------------------------------------------------------
# Step 4. LLM 연결 - Google Gemini 설정
# -----------------------------------------------------------
# LLM(Large Language Model): 텍스트를 이해하고 자연스러운 답변을 생성하는 대형 AI
# Gemini: 구글이 만든 LLM. API를 통해 구글 서버에 요청을 보내 답변을 받습니다.


gemini_client = google_genai.Client(api_key=os.environ["GOOGLE_API_KEY"])


# 사용 가능한 Gemini 모델을 순서대로 시도합니다.
# 계정 등급/지역/할당량에 따라 사용 가능한 모델이 다를 수 있습니다.
#   gemini-2.0-flash-lite : 가장 빠르고 가벼움 (무료 분당 15회 제공)
#   gemini-2.0-flash-001  : 중간 성능
#   gemini-2.5-flash      : 가장 강력하지만 무료 할당량이 적음 (분당 5회)


GEMINI_MODEL = None
for candidate in ["gemini-2.0-flash-lite", "gemini-2.0-flash-001", "gemini-2.5-flash"]:
    try:
        gemini_client.models.generate_content(model=candidate, contents="test")  # 테스트 호출
        GEMINI_MODEL = candidate                                   # 성공하면 이 모델을 사용
        print(f"LLM 모델: {candidate}")
        break                                                      # 성공 후 더 이상 시도하지 않음
    except Exception:                         
        pass                                                       # 실패하면 다음 모델 시도


# -----------------------------------------------------------
# Step 5. 프롬프트 템플릿 정의
# -----------------------------------------------------------
# 프롬프트(Prompt): LLM에게 보내는 지시문 + 질문의 조합
# PromptTemplate: {변수명} 자리에 실제 값을 채워 완성된 프롬프트를 만드는 틀
#   {context} = 벡터DB에서 검색된 관련 문서 조각 (자동으로 채워짐)
#   {query}   = 사용자가 입력한 질문 (자동으로 채워짐)

# 기본 프롬프트: 규칙 없이 단순하게 묻는 버전


basic_prompt = PromptTemplate(
    template="다음 문서를 참고하여 질문에 답하세요.\n\n문서 내용: {context}\n\n질문: {query}\n\n답변:",
    input_variables=["context", "query"]
)

# 커스텀 프롬프트: 답변 규칙이 추가된 전문가용 버전
# 규칙을 추가할수록 할루시네이션(AI가 정보를 지어내는 현상)이 줄어듭니다.


custom_prompt = PromptTemplate(
    template="""당신은 제조업 기술문서 전문 AI 어시스턴트입니다.
다음 문서 내용만을 참고하여 질문에 정확하게 답하세요.

문서 내용:
{context}

질문: {query}

답변 규칙:
1. 반드시 문서에 있는 정보만 사용하세요
2. 문서에 없으면 "해당 정보를 제공된 문서에서 찾을 수 없습니다"라고 답하세요
3. 수치는 단위를 반드시 포함하세요
4. 가능하면 출처 페이지를 언급하세요

답변:""",
    input_variables=["context", "query"]
)


# -----------------------------------------------------------
# Step 6. RAG 실행 함수 정의
# -----------------------------------------------------------


def ask(query):
    # [데모용 핵심 함수] 질문 하나로 RAG 전체 파이프라인을 실행합니다.
    #
    # 실행 흐름 (RAG 5단계):
    #   1. 질문을 임베딩(숫자 벡터)으로 변환
    #   2. 벡터DB에서 유사한 문서 조각 3개 검색
    #   3. 검색된 조각들을 하나의 컨텍스트 텍스트로 합치기
    #   4. [컨텍스트 + 질문]을 Gemini에 전달 -> 답변 생성
    #   5. 답변 + 출처를 HTML 카드 형태로 출력

    # [1~2단계] 질문과 의미가 가장 비슷한 문서 조각 3개 검색

    docs = retriever.invoke(query)

    # [3단계] 검색된 조각들을 하나의 긴 텍스트(컨텍스트)로 합치기

    context = "\n\n".join(doc.page_content for doc in docs)

    # [4단계] 프롬프트 완성 후 Gemini에 전송 -> 답변 텍스트 수신

    prompt_text = custom_prompt.format(context=context, query=query)
    answer = gemini_client.models.generate_content(
        model=GEMINI_MODEL,
        contents=prompt_text
    ).text                                        # .text: 응답 객체에서 텍스트 부분만 추출

    # [5단계] 출처 정보 정리 (중복 제거)

    sources = []
    seen = set()                                  # 이미 처리한 출처를 기억 (집합 -> 중복 자동 방지)
    for doc in docs:
        src = doc.metadata.get("source", "알 수 없음").split("/")[-1]  # 파일명만 추출
        page = doc.metadata.get("page", -1) + 1   # 0-based 인덱스 -> 1-based 페이지 번호
        key = f"{src}_p{page}"
        if key not in seen:
            seen.add(key)
            snippet = doc.page_content[:80].replace("\n", " ").strip()  # 미리보기 80자
            sources.append((src, page, snippet))


    # 출처 카드 HTML 생성

    src_html = ""
    for i, (src, page, snippet) in enumerate(sources, 1):
        src_html += f"""
        <div style='margin:4px 0; padding:6px 12px; background:#f0e6ff; border-radius:6px; font-size:13px;'>
        <b>[{i}]</b> {src} - <b>p.{page}</b>
        <br><span style='color:#8888aa; font-size:11px;'>"...{snippet}..."</span>
        </div>"""


    # 답변 + 출처를 예쁜 HTML 카드로 출력

    html = f"""
    <div style='max-width:800px; font-family:Arial,sans-serif;'>
    <div style='background:#4A1A6B; color:white; padding:12px 20px; border-radius:10px 10px 0 0; font-size:14px;'>
    <b>Q:</b> {query}
    </div>
    <div style='background:white; border:2px solid #E8DEF8; padding:16px 20px;'>
    <div style='font-size:15px; line-height:1.6; color:#1A1A2E;'>
    <b>답변:</b><br>{answer}
    </div>
    </div>
    <div style='background:#f5f1fa; padding:12px 20px; border-radius:0 0 10px 10px; border:1px solid #E8DEF8; border-top:none;'>
    <div style='font-size:13px; color:#7B2FBE; font-weight:bold; margin-bottom:6px;'>참조 출처:</div>
    {src_html}
    </div>
    </div>
    """
    display(HTML(html))


def compare(query):
    # [비교 데모용] 같은 질문을 기본 프롬프트 vs 커스텀 프롬프트에 동시에 보내 비교.
    # 수업 포인트: 프롬프트 규칙 하나가 답변 품질을 어떻게 바꾸는지 보여줍니다.

    # 동일한 문서 조각으로 두 프롬프트를 비교해야 공정합니다

    docs = retriever.invoke(query)
    context = "\n\n".join(doc.page_content for doc in docs)


    # 기본 프롬프트로 Gemini 호출 (규칙 없음)

    r_basic = gemini_client.models.generate_content(
        model=GEMINI_MODEL,
        contents=basic_prompt.format(context=context, query=query)
    ).text


    # 커스텀 프롬프트로 Gemini 호출 (규칙 4가지 포함)

    r_custom = gemini_client.models.generate_content(
        model=GEMINI_MODEL,
        contents=custom_prompt.format(context=context, query=query)
    ).text


    # 두 답변을 나란히 보여주는 HTML 출력
    
    html = f"""
    <div style='max-width:800px; font-family:Arial,sans-serif;'>
    <div style='background:#4A1A6B; color:white; padding:12px 20px; border-radius:10px 10px 0 0; font-size:14px;'>
    <b>Q:</b> {query}
    </div>
    <div style='display:flex; gap:0;'>
    <div style='flex:1; background:#fff; border:2px solid #ccc; padding:14px; border-radius:0 0 0 10px;'>
    <div style='background:#8888aa; color:white; padding:4px 10px; border-radius:4px; font-size:12px; font-weight:bold; margin-bottom:8px; text-align:center;'>기본 체인</div>
    <div style='font-size:14px; line-height:1.5; color:#4A4A6A;'>{r_basic}</div>
    </div>
    <div style='flex:1; background:#fff; border:2px solid #7B2FBE; padding:14px; border-radius:0 0 10px 0;'>
    <div style='background:#7B2FBE; color:white; padding:4px 10px; border-radius:4px; font-size:12px; font-weight:bold; margin-bottom:8px; text-align:center;'>커스텀 체인</div>
    <div style='font-size:14px; line-height:1.5; color:#1A1A2E; font-weight:500;'>{r_custom}</div>
    </div>
    </div>
    </div>
    """
    display(HTML(html))


print("")
print("시스템 준비 완료!")
print("-" * 40)
print("ask(\"질문\")     -> 답변 + 출처 카드 출력")
print("compare(\"질문\") -> 기본 vs 커스텀 나란히 비교")

---
---
# 출처 검증 데모 시작

> **여기부터 수강생에게 보여줍니다.**
>
> 블록1에서 `ask("질문")`의 답변을 봤습니다.
> 이번에는 **출처가 정말 맞는지** 같이 확인합니다.
>
> **수강생에게**: 사양서 PDF와 시험성적서 PDF를 열어두세요.

## 데모 ① 사양서 질문 → PDF에서 출처 대조

In [3]:
ask("RF9000의 소비전력은 얼마인가요?")

> **사양서 p.2**를 열어보세요. 소비전력 36W가 실제로 있나요?

In [ ]:
ask("E3 에러코드는 무엇인가요? 원인과 조치 방법을 알려주세요.")

> **사양서 p.4** 에러코드표에서 E3을 찾아보세요. 일치하나요?

In [ ]:
ask("냉장실과 냉동실의 용량 차이는 얼마인가요?")

> 계산이 필요한 질문입니다. **사양서 p.2**에서 냉장 524L, 냉동 344L → 차이 180L이 맞는지 검증해보세요.

## 데모 ② 시험성적서 질문 + 교차 참조

In [4]:
ask("에너지효율 등급과 월간 소비전력량은?")

> 출처가 **시험성적서**로 나왔나요? 사양서가 아닙니다. **시험성적서 p.2**에서 확인해보세요.

In [ ]:
ask("이 냉장고의 소비전력과 에너지효율 등급을 함께 알려주세요")

> **핵심!** 소비전력은 사양서, 에너지효율은 시험성적서 — 출처에 두 문서가 함께 나오면 **교차 참조**가 된 겁니다.

## 데모 ③ 문서에 없는 질문을 하면?

In [5]:
ask("RF9000의 판매 가격은 얼마인가요?")

> 가격은 사양서에도 시험성적서에도 없습니다.
> LLM만 쓰면 "약 200만원" 같은 답을 지어내지만, RAG는 "찾을 수 없다"고 합니다.
> 이것이 **할루시네이션 방지** — 내일 Day4에서 더 깊이 다룹니다.

In [ ]:
ask("RF9000과 RF8000의 성능 차이는?")

> RF8000은 문서에 존재하지 않는 제품입니다. 어떻게 답했나요?

## 데모 ④ 수강생 자유 질문

> **수강생에게 질문을 받아서 아래 셀에 타이핑하세요.**
>
> 질문 후 함께 PDF를 열어서 출처가 맞는지 대조합니다.

In [ ]:
ask("")

In [ ]:
ask("")

In [ ]:
ask("")

---
## 강사 전용: 추가 질의 모음

> 시간이 남거나 수강생 질문에 대응할 때 사용

In [6]:
# 교차 참조 질문
ask("IEC 60335 기준으로 절연저항은 적합한가요?")

In [ ]:
# 긴 답변 — 에러코드 전체 목록
ask("E1부터 E9까지 모든 에러코드의 명칭을 알려주세요.")

In [ ]:
# 기능 관련
ask("FoodAI 식품인식 시스템의 인식 정확도는?")

In [ ]:
# 소음 수준 — 시험성적서
ask("냉장 운전 소음 측정값은?")

In [ ]:
# 유지보수 관련
ask("탈취 필터 교체 주기는 얼마인가요?")

In [ ]:
# 복합 교차 질문
ask("이 냉장고의 크기, 무게, 그리고 냉각 성능 시험 결과를 알려주세요")

In [ ]:
# EMC 관련
ask("전도성 방출 시험 결과가 기준 이내인가요?")